In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [2]:
from pathlib import Path
from birdnet import SpeciesPredictions, Species
from birdnet import SpeciesPredictions, predict_species_within_audio_file,predict_species_within_audio_files_mp,predict_species_at_location_and_time
import pandas as pd
from glob import glob
from collections import OrderedDict
from matplotlib import pyplot as plt
import soundfile as sf
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
import torch
tf.experimental.numpy.experimental_enable_numpy_behavior()

2025-04-09 12:46:37.954726: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-09 12:46:37.996641: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-09 12:46:37.996661: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-09 12:46:37.998274: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-09 12:46:38.006164: I tensorflow/core/platform/cpu_feature_guar

In [3]:
ebird_name = pd.read_csv('/root/projects/BirdClef2025/data/train.csv')
scientific_name_common_name = list(ebird_name['scientific_name'] +'_'+ ebird_name['common_name'])
ebird_name_list = list(ebird_name['primary_label'])
ebird_birdclef2025 = set(ebird_name['primary_label'])
species_in_area = predict_species_at_location_and_time(6.76, -74.21)
for ebird,item in zip(ebird_name_list.copy(),scientific_name_common_name.copy()):
    if item not in species_in_area.keys():
        scientific_name_common_name.remove(item)
        ebird_name_list.remove(ebird)
ebird_scientific_common = pd.DataFrame({'ebird':ebird_name_list,'scientifica_common':scientific_name_common_name})
ebird_scientific_common = ebird_scientific_common.drop_duplicates()

2025-04-09 12:46:45.279777: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21965 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:3b:00.0, compute capability: 8.6
2025-04-09 12:46:46.261695: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [ ]:
filelists = glob('/root/projects/BirdClef2025/data/train_audio/anhing/*.ogg')
filelists.sort()
predictions = SpeciesPredictions(predict_species_within_audio_file(Path('/root/projects/BirdClef2025/data/train_soundscapes_20s/H98_20230518_055000_40s.ogg'),species_filter=set(scientific_name_common_name)))

In [5]:
bc_labels = pd.read_csv('/root/projects/BirdClef2025/bvc_model/assets/label.csv').iloc[:, 0].to_list()
bc_labels_indices = range(len(bc_labels))

primary_labels_map = dict(zip(bc_labels, bc_labels_indices))
bvc_classes = [pl for pl in ebird_birdclef2025 if pl in bc_labels]
bn_clasees = list(ebird_scientific_common['ebird'])

birdclassifier_last = len(bc_labels)
birdclassifier_indices = [primary_labels_map[i] for i in ebird_birdclef2025 if i in primary_labels_map]

print(f'Note: we can predict {len(birdclassifier_indices)} species only!')

Note: we can predict 143 species only!


In [6]:
model = hub.load('/root/projects/BirdClef2025/bvc_model')

# Input: 5 seconds of silence as mono 32 kHz waveform samples.
waveform = sf.read('/root/projects/BirdClef2025/data/train_audio/brtpar1/XC449467.ogg')[0][:5*32000]
waveform = waveform[None,:].repeat(repeats=32,axis=0)
waveform = tf.convert_to_tensor(waveform, dtype=tf.float32)
# Run the model, check the output.
model_outputs = model.infer_tf(waveform)['label']
prob = tf.nn.sigmoid(model_outputs)

2025-04-09 12:47:07.095059: I external/local_xla/xla/service/service.cc:168] XLA service 0x112deda0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-04-09 12:47:07.095090: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2025-04-09 12:47:07.363544: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-04-09 12:47:07.372210: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator jax2tf_infer_fn_/assert_equal_1/Assert/AssertGuard/Assert
I0000 00:00:1744202837.293430 3104654 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1744202837.346914 3104654 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [140]:
from concurrent.futures import ThreadPoolExecutor
audio_list = glob('/root/projects/BirdClef2025/data/train_soundscapes/*')
model = hub.load('/root/projects/BirdClef2025/bvc_model')
def pred_bvc(audios):
    waveform,_ = sf.read(audios)
    if len(waveform.shape)!=1:
        waveform = waveform[0,:]
    waveform = waveform.reshape(12,-1)
    model_outputs = model.infer_tf(waveform)['label']
    prob = tf.nn.sigmoid(model_outputs).numpy()
    audionames = [audios.replace('.ogg',f'_{(i+1)*5}.ogg') for i in range(prob.shape[0])]
    return prob,audionames

audionames_list = []
probs_list = []
with ThreadPoolExecutor(max_workers=8) as executor:
    for prob,audionames in executor.map(pred_bvc, audio_list):
        audionames_list += [audionames]
        probs_list += [prob]

2025-04-05 12:22:11.557680: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator jax2tf_infer_fn_/assert_equal_1/Assert/AssertGuard/Assert


In [4]:
import torch
demo = torch.tensor([2,1,2]).to(torch.float32)
print(torch.log_softmax(demo, dim=0))
print(torch.log(torch.softmax(demo, dim=0)))

tensor([-0.8620, -1.8620, -0.8620])
tensor([-0.8620, -1.8620, -0.8620])


In [191]:
audionames_lists = []
probs_lists = []
for audio in audionames_list:
    audionames_lists+=audio
for prob in probs_list:
    probs_lists+=list(prob)
probs_array = np.array(probs_lists)

In [ ]:
bvc_pred_df = pd.DataFrame({'audios':audionames_lists})
for name,index in zip(bvc_classes,birdclassifier_indices):
    bvc_pred_df[name] = probs_array[:,index]
# bvc_pred_df['probs_max'] = bvc_pred_df['probs'].map(lambda x:x.max())
bvc_pred_df.to_csv('/root/projects/BirdClef2025/BirdCLEF2023-30th-place-solution-master/usefulFunc/pesudo_labelv7_bvc.csv',index=False)

In [336]:
from tqdm import tqdm

bvc_pred_df['filename'] = bvc_pred_df['audios'].map(lambda x:'_'.join(x.split('/')[-1].split('_')[:-1])+'.ogg')
bvc_pred_df_10 = pd.DataFrame(columns=bvc_pred_df.columns)
for name,group in tqdm(bvc_pred_df.groupby('filename'),total=len(bvc_pred_df['filename'].unique())):
    for step in range(0,12,2):
        bvc_pred_df_10.loc[len(bvc_pred_df_10.index)] = [name.replace('.ogg',f'_{step*5}s.ogg')] + list(group[bvc_classes].iloc[step:step+2].max().values) + [name]
bvc_pred_df_10 = bvc_pred_df_10.drop('filename',axis=1)
bvc_pred_df_10['path'] = bvc_pred_df_10['audios'].map(lambda x:'/root/projects/BirdClef2025/data/train_soundscapes_10s/'+x)
bvc_pred_df_10 = bvc_pred_df_10.drop('audios',axis=1)
bvc_pred_df_10.to_csv('/root/projects/BirdClef2025/BirdCLEF2023-30th-place-solution-master/usefulFunc/pesudo_label_bvc10.csv')

100%|██████████| 9726/9726 [17:43<00:00,  9.15it/s]


In [4]:
# ensemble pesudo_label
efv2b3_pred = pd.read_csv('/root/projects/BirdClef2025/BirdCLEF2023-30th-place-solution-master/usefulFunc/pesudo_label_efv2b3_10.csv')
seresnext_pred = pd.read_csv('/root/projects/BirdClef2025/BirdCLEF2023-30th-place-solution-master/usefulFunc/pesudo_label_seresnext_10.csv')
nfnetl0_pred = pd.read_csv('/root/projects/BirdClef2025/BirdCLEF2023-30th-place-solution-master/usefulFunc/pesudo_label_nfnetl0_10.csv')

col_list = []
pred_dict = {'path':list(efv2b3_pred['path'])}
agree_dict = {'path':list(efv2b3_pred['path'])}
for col in list(efv2b3_pred.columns)[1:]:
    pred_max = np.max(np.array([efv2b3_pred[col].values , seresnext_pred[col].values , nfnetl0_pred[col].values]),axis=0)
    pred_in = np.min(np.array([efv2b3_pred[col].values , seresnext_pred[col].values  , nfnetl0_pred[col].values]),axis=0)
    pred_error = np.abs(pred_max - pred_in)
    pred_dict[col] = (efv2b3_pred[col].values + seresnext_pred[col].values+nfnetl0_pred[col].values)/3
    agree_dict[col] = pred_error<0.3

In [5]:
ensemble_mask_df = pd.DataFrame(pred_dict)
ensemble_mask_df.to_csv('/root/projects/BirdClef2025/BirdCLEF2023-30th-place-solution-master/usefulFunc/pesudo_labelv12_ensemble.csv',index=False)

In [ ]:
pred_std = np.std(np.array([efv2b3_pred[col].values , seresnext_pred[col].values , bvc_pred[col].values , nfnetl0_pred[col].values]),axis=0)
np.where((pred_std==0.05))[0]

array([    0,     1,     2, ..., 58353, 58354, 58355])

In [ ]:
from transformers import EfficientNetForImageClassification
import torch
import torchaudio
from torchvision import transforms
import requests
import torchaudio
import io
import soundfile as sf


audio, sample_rate = sf.read('/root/projects/BirdClef2025/externaldata/birdclef-2024/train_audio/aspfly1/XC633461.ogg')
print("Original shape and sample rate: ", audio.shape, sample_rate)
# crop to 5 seconds
audio = torch.from_numpy(audio[:int(5*sample_rate)])
# resample to 32kHz
resample = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=32000)
audio = resample(audio)
print("Resampled shape and sample rate: ", audio.shape, 32000)


CACHE_DIR = "../../data_birdset"  # Change this to your own cache directory

# Load the model
model = EfficientNetForImageClassification.from_pretrained(
    "DBD-research-group/EfficientNet-B1-BirdSet-XCL",
    num_channels=1,
    cache_dir=CACHE_DIR,
    ignore_mismatched_sizes=True,
)


class PowerToDB(torch.nn.Module):
    """
    A power spectrogram to decibel conversion layer. See birdset.datamodule.components.augmentations
    """

    def __init__(self, ref=1.0, amin=1e-10, top_db=80.0):
        super(PowerToDB, self).__init__()
        # Initialize parameters
        self.ref = ref
        self.amin = amin
        self.top_db = top_db

    def forward(self, S):
        # Convert S to a PyTorch tensor if it is not already
        S = torch.as_tensor(S, dtype=torch.float32)

        if self.amin <= 0:
            raise ValueError("amin must be strictly positive")

        if torch.is_complex(S):
            magnitude = S.abs()
        else:
            magnitude = S

        # Check if ref is a callable function or a scalar
        if callable(self.ref):
            ref_value = self.ref(magnitude)
        else:
            ref_value = torch.abs(torch.tensor(self.ref, dtype=S.dtype))

        # Compute the log spectrogram
        log_spec = 10.0 * torch.log10(
            torch.maximum(magnitude, torch.tensor(self.amin, device=magnitude.device))
        )
        log_spec -= 10.0 * torch.log10(
            torch.maximum(ref_value, torch.tensor(self.amin, device=magnitude.device))
        )

        # Apply top_db threshold if necessary
        if self.top_db is not None:
            if self.top_db < 0:
                raise ValueError("top_db must be non-negative")
            log_spec = torch.maximum(log_spec, log_spec.max() - self.top_db)

        return log_spec


def preprocess(audio, sample_rate_of_audio):
    """
    Preprocess the audio to the format that the model expects
    - Resample to 32kHz
    - Convert to melscale spectrogram n_fft: 2048, hop_length: 256, power: 2. melscale: n_mels: 256, n_stft: 1025
    - Normalize the melscale spectrogram with mean: -4.268, std: 4.569 (from AudioSet)

    """
    powerToDB = PowerToDB()
    # Resample to 32kHz
    resample = torchaudio.transforms.Resample(
        orig_freq=sample_rate_of_audio, new_freq=32000
    )
    audio = resample(audio)
    spectrogram = torchaudio.transforms.Spectrogram(
        n_fft=2048, hop_length=256, power=2.0
    )(audio)
    melspec = torchaudio.transforms.MelScale(n_mels=256, n_stft=1025)(spectrogram)
    dbscale = powerToDB(melspec)
    normalized_dbscale = transforms.Normalize((-4.268,), (4.569,))(dbscale)
    return normalized_dbscale

preprocessed_audio = preprocess(audio.to(torch.float32).unsqueeze(0), sample_rate)
print("Preprocessed_audio shape:", preprocessed_audio.shape)

logits = model(preprocessed_audio.unsqueeze(0)).logits
print("Logits shape: ", logits.shape)

top5 = torch.topk(logits, 5)
print("Top 5 logits:", top5.values)
print("Top 5 probs:", top5.values)
print("Top 5 predicted classes:")
print([model.config.id2label[i] for i in top5.indices.squeeze().tolist()])
